In [1]:
import sagemaker
import boto3
from sagemaker import get_execution_role

In [2]:
region = boto3.Session().region_name
sm_rt = boto3.Session().client('runtime.sagemaker', region_name=region)

ep_name = 'kdd-clean-model'
id_name = ''
target = 'is_exciting_20'

file_name = "kdd-clean-test.csv"
test_file_name = "kdd-clean-test-no-label.csv"

kaggle_file_name = ""
kaggle_test_file_name = ""
kaggle_pred_file_name = ""

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv(file_name)
df = df.drop([target],axis=1) 
print(df.head())
print(df.shape)

df.to_csv(test_file_name, index=False)

  secondary_focus_subject primary_focus_subject  school_latitude    cost  \
0           Special Needs  Literature & Writing        40.461702  138.64   
1         Performing Arts                 Music        32.465762  119.98   
2             Mathematics      Applied Sciences        40.819389  198.58   
3             Mathematics  Literature & Writing        40.832025  125.33   
4             Mathematics  Literature & Writing        35.349696  254.24   

   school_longitude  Vol_teacher_acctid resource_type  \
0        -79.827803                   7    Technology   
1        -84.888363                  22         Other   
2        -73.957037                   6      Supplies   
3        -73.906682                   8      Supplies   
4       -119.162331                   2         Books   

  teacher_teach_for_america  Vol_school_city  school_zip  ...  \
0                         f              310         949  ...   
1                         f             1495        4057  ...   
2    

In [4]:
f = open(test_file_name)
count = 0
pred_label = []
pred_neg_prpb = []
pred_pos_prpb = []

for line in f:
    if count==0:
        count=1
        continue
    response = sm_rt.invoke_endpoint(EndpointName=ep_name, ContentType='text/csv', Accept='text/csv', Body=line.encode('utf-8'))
    response = response['Body'].read().decode("utf-8")
    #print(response)
    pred_label.append(int(response[0]))
    ind1 = response.index("[")
    ind2 = response.index(",", ind1)
    neg_prob = float(response[ind1+1:ind2])
    #print(neg_prob)
    pred_neg_prpb.append(neg_prob)
    ind1 = response.index("]", ind2)
    pos_prob = float(response[ind2+2:ind1])
    #print(pos_prob)
    pred_pos_prpb.append(pos_prob)
    count = count + 1


In [7]:
print(pred_label[:20])
print(pred_neg_prpb[:20])
print(pred_pos_prpb[:20])


import pandas as pd
import numpy as np
pd = pd.read_csv(file_name)
y_true = np.array(pd[target])
y_pred = np.array(pred_label)
print("test accuracy = {}".format(np.mean(y_true==y_pred)))

from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(y_true, pred_pos_prpb, pos_label=1)
print("test auc = {}".format(metrics.auc(fpr, tpr)))

[0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0.5607670545578003, 0.6581588685512543, 0.558429479598999, 0.6424836218357086, 0.4627841114997864, 0.5449536740779877, 0.615396648645401, 0.49058228731155396, 0.3542155623435974, 0.6272209584712982, 0.6416670680046082, 0.5768544673919678, 0.7577802985906601, 0.6355038285255432, 0.6191684603691101, 0.6177190542221069, 0.624062716960907, 0.41109442710876465, 0.5625975131988525, 0.6359972953796387]
[0.4392329454421997, 0.3418411314487457, 0.441570520401001, 0.3575163781642914, 0.5372158885002136, 0.45504632592201233, 0.384603351354599, 0.509417712688446, 0.6457844376564026, 0.3727790415287018, 0.35833293199539185, 0.4231455326080322, 0.2422197014093399, 0.3644961714744568, 0.3808315396308899, 0.38228094577789307, 0.375937283039093, 0.5889055728912354, 0.43740248680114746, 0.36400270462036133]
test accuracy = 0.7740986019131715
test auc = 0.7244503026494227


In [6]:
import pandas as pd
import numpy as np

df = pd.read_csv(kaggle_file_name)

kaggle_test_ids = np.array(df[id_name])

df = df.drop([id_name],axis=1) 
print(df.head())
print(df.shape)

df.to_csv(kaggle_test_file_name, index=False)

FileNotFoundError: [Errno 2] File  does not exist: ''

In [14]:
f = open(kaggle_test_file_name)
count = 0
kaggle_pred = []

for line in f:
    if count == 0:
        count = 1
        continue
    response = sm_rt.invoke_endpoint(EndpointName=ep_name, ContentType='text/csv', Accept='text/csv', Body=line.encode('utf-8'))
    response = response['Body'].read().decode("utf-8")
    test_id = kaggle_test_ids[count-1]
    pred = int(response[0])
    kaggle_pred.append([test_id, pred])
    count = count + 1

kaggle_pred_df = pd.DataFrame(kaggle_pred, columns = [id_name, target])

kaggle_pred_df.to_csv(kaggle_pred_file_name, index=False)

In [5]:
print(kaggle_pred_df.head())
print(kaggle_pred_df.shape)

NameError: name 'kaggle_pred_df' is not defined

In [22]:
import boto3
sm = boto3.client('sagemaker')
#sm.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm.delete_endpoint(EndpointName=ep_name)

{'ResponseMetadata': {'RequestId': '3adbc464-12f4-4c5b-af4a-1bad8f737f35',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3adbc464-12f4-4c5b-af4a-1bad8f737f35',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Wed, 24 Mar 2021 07:51:25 GMT'},
  'RetryAttempts': 0}}